Assignment 4

Code
1. Admin stuff: Loading all libraries and data
2. data preprocessing
3. Kaplan-Meier Analysis
4. Cox Proportional Hazards Regression
5. Random Survival Forests (RSF)

Things to improve: 
- RSF runs for a long time which makes me thing i didn't close some loop. I fixed a lot of minor errors but it's hard to know what to fix when it never stops running. 
- I didn't compare the RSF model’s concordance index (C-index) with that of Cox regression.
- Adding training and testing split.
- Adding better annotations.


References (all accessed July 15-17.2025):

- https://lifelines.readthedocs.io/en/latest/lifelines.statistics.html - multivariate_logrank_test

- https://scikit-survival.readthedocs.io/en/stable/user_guide/random-survival-forest.html - random survival forest

AI was used for a lot of troubleshooting and pointing to resources mostly for RSF. I looked at code examples online, more than I pasted as references. Some annotations. Probably help with readme but I didn't write it yet.

In [ ]:
### Admin stuff ###
import data_preprocessor as dp
import matplotlib.pyplot as plt
import pandas as pd
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
from lifelines import CoxPHFitter
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv
from sksurv.preprocessing import OneHotEncoder as SKSurvEncoder
from sklearn.inspection import permutation_importance

### loading data ###
messy_data = pd.read_excel(r'../Data/RADCURE_Clinical_v04_20241219.xlsx', na_values=['', 'NA', 'Na', 'null', ""])
working_data = messy_data.copy()
#working_data.to_csv(r"../Data/cleaning_step0.csv", index=False)

All data exploration was done in data wrangler extension of vscode.

I followed the usual data cleaning steps: dropping duplicates, removing columns and rows with 25+% missing data, dropping the rows with other missing data ("subsite" column had 11% of missing data. I decided to delete those too although I could have added ex "unknown" value instead. But at the end I still had almost 3k rows so I decided on deleting those rows)

Finally I used "Status" column to make a binary "Event" column for survival. And I defined duration and event columns.

Final stats: messy data = 3,346 rows x 34 columns, clean data = 2,946 rows x 24 columns

In [ ]:
### cleaning the data ###
working_data = working_data.drop_duplicates()
working_data = dp.remove_columns_missing_data(working_data)
working_data = dp.remove_rows_missing_data(working_data)
#working_data.to_csv(r"../Data/cleaning_step2.csv", index=False)

# print(working_data.columns.tolist())
columns = ["Smoking PY", "Subsite","T","N","M ","Stage"]
working_data = working_data.dropna(subset = columns)
#working_data.to_csv(r"../Data/cleaning_step3.csv", index=False)

# add event column
working_data['Event'] = working_data['Status'].map({'Dead': 1, 'Alive': 0})
working_data.to_csv(r"../Data/clean_data.csv", index=False)

clean_data = working_data

duration = "Length FU"
event = "Event"

In [ ]:
### Kaplan-Meier Analysis ###
# 2 groups chosen: Stage and Chemo

kmf = KaplanMeierFitter()

## KM curves by stage
for stage, group in clean_data.groupby("Stage"):
    kmf.fit(group[duration], event_observed = group[event], label = f"Stage {stage}")
    kmf.plot_survival_function()

# Plot the Kaplan-Meier curve
kmf.plot_survival_function()
plt.title('Kaplan-Meier Curve')
plt.xlabel('Time (weeks)')
plt.ylabel('Survival Probability')
plt.show()

# multivariate_logrank_test for stage comparison
result = multivariate_logrank_test(
    event_durations=clean_data[duration],
    groups=clean_data["Stage"],
    event_observed=clean_data[event])
result.print_summary()

## KM curves by chemo
for chemo, group in clean_data.groupby("Chemo"):
    kmf.fit(group[duration], event_observed = group[event], label = chemo)
    kmf.plot_survival_function()

# Plot the Kaplan-Meier curve
kmf.plot_survival_function()
plt.title('Kaplan-Meier Curve')
plt.xlabel('Time (weeks)')
plt.ylabel('Survival Probability')
plt.show()

# multivariate_logrank_test for chemo comparison. I chose to use this one for speed even though it's binary.
result = multivariate_logrank_test(
    event_durations=clean_data[duration],
    groups=clean_data["Chemo"],
    event_observed=clean_data[event])
result.print_summary()

In [ ]:
### Cox Proportional Hazards Regression ###

# convert to binary and dummy
covariates = ["Chemo", "Sex", "Stage"]
cox_data = clean_data[[duration, event] + covariates].copy()
cox_data["Chemo"] = cox_data["Chemo"].map({"Yes": 1, "none": 0})
cox_data["Sex"] = cox_data["Sex"].map({"Female": 1, "Male": 0})
cox_data = pd.get_dummies(cox_data, columns=["Stage"], drop_first=True)
cox_data = cox_data.drop(columns=["Stage_IB", "Stage_IIA", "Stage_IVC"]) # made the plot unreadable

# CPH model
cph = CoxPHFitter()
cph.fit(cox_data,
        duration_col=duration, event_col=event)
cph.print_summary()

# plot 
cph.plot()
plt.title('Cox Regression Coefficients')
plt.show()

# check assumptions
cph.check_assumptions(cox_data, p_value_threshold=0.05, show_plots=True)
# Stage_IVB failed the assumption

In [ ]:
### Random Survival Forests (RSF) ###

# prep data
rsf_data = clean_data[[duration, event] + covariates].copy()
rsf_data["Chemo"] = rsf_data["Chemo"].map({"Yes": 1, "none": 0})
rsf_data["Sex"] = rsf_data["Sex"].map({"Female": 1, "Male": 0})
rsf_data = rsf_data.select_dtypes(exclude=["datetime64[ns]"])

data_x = rsf_data.drop(columns=[duration, event], axis=1)
data_y = Surv.from_dataframe(event=event, time=duration, data=rsf_data)
for col in data_x.select_dtypes(include="object").columns:
    data_x[col] = data_x[col].astype("category")

encoder = SKSurvEncoder()
data_x = encoder.fit_transform(data_x)

# survival model
rsf = RandomSurvivalForest(n_estimators=100, random_state=42)
rsf.fit(data_x, data_y)

result = permutation_importance(rsf, data_x, data_y, n_repeats=15, random_state=42)
feature_importance = pd.DataFrame(
         {
        k: result[k]
        for k in (
            "importances_mean",
            "importances_std",
        )
    },
    index=data_x.columns,
).sort_values(by="importances_mean", ascending=False)

# Sort by importances_mean and plot
feature_importance = feature_importance.sort_values(by="importances_mean", ascending=False)

plt.figure(figsize=(10, 6))
plt.title('Feature Importances')
plt.barh(feature_importance.index, feature_importance['importances_mean'], xerr=feature_importance['importances_std'], align='center')
plt.xlabel('Mean Importance')
plt.ylabel('Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()